# Basic of asynchronous inferencing
It is very important to utilize asynchronous inferencing capability of OpenVINO to make your application efficient and high performance.  
The `infer()` API used in the synchronous inferencing is a blocking function and the program will be kept waiting until inferencing is completed. So, the other task can't be performed in parallel.

### Preparing an input image and label text data files
First, let's prepare imput image file and class label text file. Those files are in the OpenVINO install directory. We'll simply copy them to the current working directory.

In [ ]:
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/squeezenet1.1.labels synset_words.txt

Show the image file and check the picture.

In [ ]:
from IPython.display import Image
Image('car.png')

### Preparing a DL model for inferencing
Download a DL model for image classification using `Model downloader` and convert it into OpenVINO IR model with `Model converter`.  
We'll use `googlenet-v1` model for this practice.

In [ ]:
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name googlenet-v1
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name googlenet-v1 --precisions FP16
!ls public/googlenet-v1/FP16 -l

----
The Python inferencing code starts from here.

### Initialize application for OpenVINO
This part is identical to the program in the previous image classification exercise. 
1. Import required Python modules
2. Load class label text file
3. Create an inference engine core object
4. Load IR model to memory
5. Obtain information of input and output blob
6. Load model data to inference engine core object

In [ ]:
import time

import cv2
import numpy as np
from openvino.inference_engine import IENetwork, IECore

label = open('synset_words.txt').readlines()

# Create an Inference Engine core object
ie = IECore()

# Read an IR model data to memory
model = './public/googlenet-v1/FP16/googlenet-v1'
net = IENetwork(model=model+'.xml', weights=model+'.bin')

# Obtain the name of the input and output blob, and input blob shape
input_blob_name  = list(net.inputs.keys())[0]
output_blob_name = list(net.outputs.keys())[0]
batch,channel,height,width = net.inputs[input_blob_name].shape

exec_net = ie.load_network(network=net, device_name='CPU', num_requests=1)

### Setting callback function to the infer_request object
Set a callback function to the `infer_request` object. You can use noname function (lambd espression) as well.  
The second parameter in `set_completion_callback()` API is a user data and will be passed to the callback function. In this code, `exec_net.requests[req_id]` will be passed to the callback function to display the result.  
The callback function displays the result of inferencing.

In [ ]:
def callback(status_code, infer_request):
    output = infer_request.outputs[output_blob_name][0]
    idx = np.argsort(output)[::-1]
    for i in range(5):
        print(idx[i]+1, output[idx[i]], label[idx[i]][:-1])

req_id=0
exec_net.requests[req_id].set_completion_callback(callback, exec_net.requests[req_id])

### Reading and manipulate input image
Read the input image file and resize and transform it to fit it for input blob of the DL model using OpenCV.

In [ ]:
img = cv2.imread('car.png')
img = cv2.resize(img, (width,height))
img = img.transpose((2, 0, 1))
img = img.reshape((1, channel, height, width))

### Running inference  
The `start_async()` API will kick a thread and start inferencing in parallel, and the control will be returned to your program immediately.  So, the number displaying code will run while inferencing is ongoing. The inferencing will finish in the middle of number display and the result will be displayed in between of numbers. 

In [ ]:
res = exec_net.start_async(0, inputs={input_blob_name: img})

for i in range(10):
    print(i)
    time.sleep(0.1)    # wait for 0.2 sec

----
Now, you have learnt how to use aynchronous inference API of OpenVINO.

You can use the hardware resources in very efficient manner and could achieve high performance.  
Especially it is important when you offload your inferencing task from CPU to other devices such as GPU, MYRIAD and FPGA. If you use a synchronous API to inference, the CPU will just waste processing capability and wait for completion of the inferencing task performed in the other devices.  

Get familiar with the synchronous inferencing API and develop an efficient and high performance deep learning application with OpenVINO.

## Next => Technique for high performance inference program - asynchronous and simultaneous inferencing - [classification-async-multi.ipynb](./classification-async-multi.ipynb)